In [9]:
import pandas as pd
import boto3
import io
from datetime import datetime

In [ ]:
ACCESS_KEY = ''
SECRET_KEY = ''


s3_client = boto3.client('s3', region_name='us-east-1', aws_access_key_id=ACCESS_KEY,
                         aws_secret_access_key=SECRET_KEY)
athena_client = boto3.client('athena', region_name='us-east-1', aws_access_key_id=ACCESS_KEY,
                         aws_secret_access_key=SECRET_KEY)

session = boto3.Session(aws_access_key_id=ACCESS_KEY,
                        aws_secret_access_key=SECRET_KEY)

In [11]:
source_db = 'devoted_health_prod'
QUERY_OUTPUT_LOCATION = "s3://zignaai-deidentified-claimsdata/query output/"
WORKGROUP_NAME = 'SelectionQueries-Production'

In [12]:
current_date = datetime.today().date()
LOOKBACK_DATE = (current_date - pd.DateOffset(months=18)).replace(day=1).strftime("'%Y-%m-%d'")

In [13]:
def run_query(athena_client, s3_client, query_string, source_db, QUERY_OUTPUT_LOCATION, WORKGROUP):
    
    try:
        response = athena_client.start_query_execution(
            QueryString=query_string,
            QueryExecutionContext={
                'Database': source_db
            },
            ResultConfiguration={
                'OutputLocation': QUERY_OUTPUT_LOCATION
            },
            WorkGroup=WORKGROUP
        )
        query_execution_id = response['QueryExecutionId']
        
        while True:
            query_status = athena_client.get_query_execution(QueryExecutionId=query_execution_id) 
            query_state = query_status['QueryExecution']['Status']['State'] 
            query_result_path = query_status['QueryExecution']['ResultConfiguration']['OutputLocation'] # location of query output
            if query_state in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
                break

        if query_state == 'SUCCEEDED':
            try:
                bucket = query_result_path.split('//')[1].split('/')[0]
                key = '/'.join(query_result_path.split('//')[1].split('/')[1:])
                buffer = io.BytesIO(s3_client.get_object(Bucket=bucket, Key=key)['Body'].read()) 
                df = pd.read_csv(buffer, dtype=str) 
                return df
            except Exception as e:
                print("QUERY EXECUTION SUCCESSFULL BUT GOT ERROR WHILE CONVERSION TO DATAFRAME")
                print(e)
        else:
            print("Query execution failed Status: {}".format(query_state))
            return None
    except Exception as e:
            print("GOT EXCEPTION WHILE RUNNING QUERY")
            print(e)
            return None

In [15]:
query_string = f"""SELECT distinct p1.payer_control_number,
		p1.member_medicare_id,
		p1.rendering_provider_npi,
		p1.service_date,
		array_intersect(
			array_distinct(array_agg(p1.procedure_code)),
			array_distinct(array_agg(column1))
		) as ref_intersect,
		array_intersect(
			array_distinct(array_agg(p2.procedure_code)),
			array_distinct(array_agg(column2))
		) as target_intersect
	FROM devoted_health_prod.transformed_claims AS p1
		JOIN devoted_health_prod.transformed_claims AS p2 ON p1.payer_control_number = p2.payer_control_number
		and p1.member_medicare_id = p2.member_medicare_id
		AND p1.rendering_provider_npi = p2.rendering_provider_npi
		AND p1.service_date = p2.service_date
		LEFT JOIN zigna_reference_data.medicare_ncci_ptp_edits on p1.procedure_code = medicare_ncci_ptp_edits.column1
		AND p2.procedure_code = medicare_ncci_ptp_edits.column2
	where p1.bill_type_code is null
		and p1.first_service_date >= effective_date
		and p1.first_service_date < deletion_date
		AND p1.plan_paid_amount > 0
		AND p2.plan_paid_amount >= 74
		AND modifier_filter = '0'
		AND p1.procedure_code <> p2.procedure_code
		AND medicare_ncci_ptp_edits.provider_type in ('practitioner')
		AND p1.payment_effective_date >= date({LOOKBACK_DATE})
		AND p1.is_final_paid_indicator = 1
	group by p1.payer_control_number,
		p1.member_medicare_id,
		p1.service_date,
		p1.rendering_provider_npi
	having cardinality(
			array_intersect(
				array_distinct(array_agg(p1.procedure_code)),
				array_distinct(array_agg(column1))
			)
		) >= 1
		and cardinality(
			array_intersect(
				array_distinct(array_agg(p2.procedure_code)),
				array_distinct(array_agg(column2))
			)
		) >= 1"""

referance_data = run_query(athena_client, s3_client, query_string, source_db, QUERY_OUTPUT_LOCATION, WORKGROUP_NAME)

In [16]:
referance_data

,payer_control_number,member_medicare_id,rendering_provider_npi,service_date,ref_intersect,target_intersect
0,AJX5J43FY8-1,8XF5G42YV59,1366559742,2024-06-24,[99239],[99232]
1,AJXWG9SUCG,7W89F70UJ34,1316933690,2024-02-08,[G0402],[G0439]
2,AJXZS9G7K7,6PM7W33JA77,1912941774,2024-01-17,[G0402],[G0438]
3,AJXZUZRZUH,1NU7UM6TF30,1427248970,2024-03-21,[G0402],[G0438]
4,AJXAU6UKAS-1,8K12CV8HD01,1801994157,2023-12-04,"[45380, 45385]",[00811]
5,AJXH94CSK7-1,7V40TJ7JC34,1356528210,2024-11-20,[58561],[58558]
6,AJX79FSAS4,2K67CV1YY64,1093209975,2024-09-20,[99223],[99232]
7,AJXA2E2599,7MR1G76WC93,1396848990,2025-01-30,[G0438],[G0439]
8,AJX8EJY69F,7JR0VA0JY11,1902153398,2025-01-13,[45380],[45378]
9,AJX49HGGAR-1,6MV0NP9NQ07,1427248970,2024-03-05,[G0402],[G0438]


In [23]:
df_target_ref_clean = referance_data.copy()

# ref_intersect col
df_target_ref_clean['ref_intersect'] = df_target_ref_clean['ref_intersect'].str.replace('[', '').str.replace(']', '')
df_target_ref_clean['ref_intersect'] = df_target_ref_clean['ref_intersect'].str.split(',')
df_target_ref_clean['ref_intersect'] = df_target_ref_clean['ref_intersect'].apply(lambda x: [i.strip() for i in x])

# target_intersect col
df_target_ref_clean['target_intersect'] = df_target_ref_clean['target_intersect'].str.replace('[', '').str.replace(']', '')
df_target_ref_clean['target_intersect'] = df_target_ref_clean['target_intersect'].str.split(',')
df_target_ref_clean['target_intersect'] = df_target_ref_clean['target_intersect'].apply(lambda x: [i.strip() for i in x])


In [ ]:
df = pd.read_csv(r"C:\Users\hamupratik\Downloads\6a38ebb3-0652-4d5c-bb3f-a09c5eba69b9.csv", dtype = str)

In [19]:
main_df_with_target_ref_codes = df.merge(df_target_ref_clean, on=['payer_control_number', 'member_medicare_id', 'service_date', 'rendering_provider_npi'],
               how='left')
main_df_with_target_ref_codes.shape

(166, 101)

In [29]:
main_df_with_target_ref_codes['target_intersect'] = main_df_with_target_ref_codes['target_intersect'].apply(lambda x: [''] if x!=x else x)
main_df_with_target_ref_codes['ref_intersect'] = main_df_with_target_ref_codes['ref_intersect'].apply(lambda x: [''] if x!=x else x)

In [30]:
main_df_with_target_ref_codes

,payer_control_number,adjudication_record_locator,member_record_locator,member_medicare_id,rendering_provider_npi,service_date,procedure_code,proc_modifier,proc_modifier2,proc_modifier3,...,final_claim_indicator,financial_recovery_indicator,is_dmg_claim,discharge_status_code,is_final_paid_indicator,load_date,root_payer_control_number,query_name,ref_intersect,target_intersect
0,AJXWR7FHG9-2,AJXJAA3CW6,D822RR,1CF5PR6AE85,1346283124,2023-06-14,93355,59,NaN,NaN,...,true,false,false,NaN,1,2024-01-19 12:35:00.000,AJXWR7FHG9,prof_bundle_medadv_ptp,[00560],[93355]
1,AJXWR7FHG9-2,AJXJAA3CW6,D822RR,1CF5PR6AE85,1346283124,2023-06-14,36620,NaN,NaN,NaN,...,true,false,false,NaN,1,2024-01-19 12:35:00.000,AJXWR7FHG9,prof_bundle_medadv_ptp,[00560],[93355]
2,AJXWR7FHG9-2,AJXJAA3CW6,D822RR,1CF5PR6AE85,1346283124,2023-06-14,36556,NaN,NaN,NaN,...,true,false,false,NaN,1,2024-01-19 12:35:00.000,AJXWR7FHG9,prof_bundle_medadv_ptp,[00560],[93355]
3,AJXWR7FHG9-2,AJXJAA3CW6,D822RR,1CF5PR6AE85,1346283124,2023-06-14,99291,25,NaN,NaN,...,true,false,false,NaN,1,2024-01-19 12:35:00.000,AJXWR7FHG9,prof_bundle_medadv_ptp,[00560],[93355]
4,AJXWR7FHG9-2,AJXJAA3CW6,D822RR,1CF5PR6AE85,1346283124,2023-06-14,00560,AA,P4,NaN,...,true,false,false,NaN,1,2024-01-19 12:35:00.000,AJXWR7FHG9,prof_bundle_medadv_ptp,[00560],[93355]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,AJXS6FUUGE-1,AJXR3JJ9HJ,DEY7KK,9PW7T31TW63,1962765834,2022-10-20,99497,33,NaN,NaN,...,true,false,false,NaN,1,2024-06-03 19:00:00.000,AJXS6FUUGE,prof_bundle_medadv_ptp,[G0402],[G0438]
162,AJXS6FUUGE-1,AJXR3JJ9HJ,DEY7KK,9PW7T31TW63,1962765834,2022-10-20,G0444,59,NaN,NaN,...,true,false,false,NaN,1,2024-06-03 19:00:00.000,AJXS6FUUGE,prof_bundle_medadv_ptp,[G0402],[G0438]
163,AJXS6FUUGE-1,AJXR3JJ9HJ,DEY7KK,9PW7T31TW63,1962765834,2022-10-20,G0402,NaN,NaN,NaN,...,true,false,false,NaN,1,2024-06-03 19:00:00.000,AJXS6FUUGE,prof_bundle_medadv_ptp,[G0402],[G0438]
164,AJXS6FUUGE-1,AJXR3JJ9HJ,DEY7KK,9PW7T31TW63,1962765834,2022-10-20,3075F,NaN,NaN,NaN,...,true,false,false,NaN,1,2024-06-03 19:00:00.000,AJXS6FUUGE,prof_bundle_medadv_ptp,[G0402],[G0438]


In [34]:
main_df_with_target_ref_codes['plan_paid_amount'] = main_df_with_target_ref_codes['plan_paid_amount'].astype(float)

In [35]:
main_df_with_target_ref_codes['proc_code_flag'] = main_df_with_target_ref_codes.fillna(value={'procedure_code':'not_present'}).apply(lambda x: 
        'target' if x['procedure_code'] in x['target_intersect'] and (x['plan_paid_amount']>=74) 
        else 
        ('reference' if x['procedure_code'] in x['ref_intersect'] and (x['plan_paid_amount']>0) else 'other'), axis=1)

In [36]:
main_df_with_target_ref_codes

,payer_control_number,adjudication_record_locator,member_record_locator,member_medicare_id,rendering_provider_npi,service_date,procedure_code,proc_modifier,proc_modifier2,proc_modifier3,...,financial_recovery_indicator,is_dmg_claim,discharge_status_code,is_final_paid_indicator,load_date,root_payer_control_number,query_name,ref_intersect,target_intersect,proc_code_flag
0,AJXWR7FHG9-2,AJXJAA3CW6,D822RR,1CF5PR6AE85,1346283124,2023-06-14,93355,59,NaN,NaN,...,false,false,NaN,1,2024-01-19 12:35:00.000,AJXWR7FHG9,prof_bundle_medadv_ptp,[00560],[93355],target
1,AJXWR7FHG9-2,AJXJAA3CW6,D822RR,1CF5PR6AE85,1346283124,2023-06-14,36620,NaN,NaN,NaN,...,false,false,NaN,1,2024-01-19 12:35:00.000,AJXWR7FHG9,prof_bundle_medadv_ptp,[00560],[93355],other
2,AJXWR7FHG9-2,AJXJAA3CW6,D822RR,1CF5PR6AE85,1346283124,2023-06-14,36556,NaN,NaN,NaN,...,false,false,NaN,1,2024-01-19 12:35:00.000,AJXWR7FHG9,prof_bundle_medadv_ptp,[00560],[93355],other
3,AJXWR7FHG9-2,AJXJAA3CW6,D822RR,1CF5PR6AE85,1346283124,2023-06-14,99291,25,NaN,NaN,...,false,false,NaN,1,2024-01-19 12:35:00.000,AJXWR7FHG9,prof_bundle_medadv_ptp,[00560],[93355],other
4,AJXWR7FHG9-2,AJXJAA3CW6,D822RR,1CF5PR6AE85,1346283124,2023-06-14,00560,AA,P4,NaN,...,false,false,NaN,1,2024-01-19 12:35:00.000,AJXWR7FHG9,prof_bundle_medadv_ptp,[00560],[93355],reference
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,AJXS6FUUGE-1,AJXR3JJ9HJ,DEY7KK,9PW7T31TW63,1962765834,2022-10-20,99497,33,NaN,NaN,...,false,false,NaN,1,2024-06-03 19:00:00.000,AJXS6FUUGE,prof_bundle_medadv_ptp,[G0402],[G0438],other
162,AJXS6FUUGE-1,AJXR3JJ9HJ,DEY7KK,9PW7T31TW63,1962765834,2022-10-20,G0444,59,NaN,NaN,...,false,false,NaN,1,2024-06-03 19:00:00.000,AJXS6FUUGE,prof_bundle_medadv_ptp,[G0402],[G0438],other
163,AJXS6FUUGE-1,AJXR3JJ9HJ,DEY7KK,9PW7T31TW63,1962765834,2022-10-20,G0402,NaN,NaN,NaN,...,false,false,NaN,1,2024-06-03 19:00:00.000,AJXS6FUUGE,prof_bundle_medadv_ptp,[G0402],[G0438],reference
164,AJXS6FUUGE-1,AJXR3JJ9HJ,DEY7KK,9PW7T31TW63,1962765834,2022-10-20,3075F,NaN,NaN,NaN,...,false,false,NaN,1,2024-06-03 19:00:00.000,AJXS6FUUGE,prof_bundle_medadv_ptp,[G0402],[G0438],other


In [ ]:
# return this
# main_df_with_target_ref_codes